In [1]:
import pandas as pd
import numpy as np
import csv

from matplotlib import pyplot as plt
%matplotlib inline
filepath = "../resources/food.csv.zip"
zf = zipfile.ZipFile(filepath)
food = pd.read_csv(filepath, parse_dates=True, dtype=object, delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',10)
food

OSError: File b'resources/food.csv' does not exist

In [6]:
def no_null_objects(data, columns=None):
    """
    selects rows with no NaNs
    """
    if columns is None:
        columns = data.columns
    return data[np.logical_not(np.any(data[columns].isnull().values, axis=1))]

In [7]:
def splitDataFrameList(df, target_column, separator):
    def splitListToRows(row, row_accumulator, target_column, separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

In [8]:
food_countries = splitDataFrameList(no_null_objects(food, ["countries_en", 'additives_n', 'allergens']), "countries_en", ",")
countries = food_countries["countries_en"]
countries_temp = (countries.unique())
countries_temp

array(['France', 'United Kingdom', 'Spain', 'United States', 'Belgium',
       'Netherlands', 'Ireland', 'Canada', 'Australia', 'Czech Republic',
       'Indonesia', 'Denmark', 'Norway', 'Sweden', 'Germany', 'Lebanon',
       'Finland', 'Portugal', 'Italy', 'Switzerland', 'fr:Angleterre',
       'French Polynesia', 'Romania', 'de:Scotland', 'European Union',
       'Slovenia', 'Allemagne', 'Poland', 'Peru', 'Morocco', 'Luxembourg',
       'Guadeloupe', 'French Guiana', 'Gabon', 'Tunisia', 'Martinique',
       'Réunion', 'New Caledonia', 'Saint Pierre and Miquelon', 'Greece',
       'Austria', 'Hungary', 'Russia', 'India', 'Armenia', 'China',
       'Saudi Arabia', 'Japan', 'Cyprus', 'Scotland', 'Estonia', 'Malta',
       'New Zealand', 'South Africa', 'United Arab Emirates', 'Mexico',
       'Serbia', 'Bulgaria', 'fr:Marseille-6', 'Brazil', 'Montenegro',
       'Algeria', 'Andorra', 'Iraq', 'Singapore', 'Thailand'], dtype=object)

In [9]:
food_countries['allergens'] = food_countries['allergens'].str.split(',').apply(len)
food_countries['additives_n'] = food_countries['additives_n'].astype(int)
food_countries
#countries_final = np.ndarray.sort(countries_temp)
#countries_final

,-alpha-linolenic-acid_100g,-arachidic-acid_100g,-arachidonic-acid_100g,-behenic-acid_100g,-butyric-acid_100g,-capric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-cerotic-acid_100g,-dihomo-gamma-linolenic-acid_100g,-docosahexaenoic-acid_100g,-eicosapentaenoic-acid_100g,-elaidic-acid_100g,-erucic-acid_100g,-fructose_100g,-gamma-linolenic-acid_100g,-glucose_100g,-gondoic-acid_100g,-lactose_100g,-lauric-acid_100g,-lignoceric-acid_100g,-linoleic-acid_100g,-maltodextrins_100g,-maltose_100g,-mead-acid_100g,-melissic-acid_100g,-montanic-acid_100g,-myristic-acid_100g,-nervonic-acid_100g,-oleic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-sucrose_100g,additives,additives_en,additives_n,additives_tags,alcohol_100g,allergens,allergens_en,beta-carotene_100g,bicarbonate_100g,biotin_100g,brands,brands_tags,caffeine_100g,calcium_100g,carbohydrates_100g,carbon-footprint_100g,casein_100g,categories,categories_en,categories_tags,chloride_100g,chlorophyl_100g,cholesterol_100g,chromium_100g,cities,cities_tags,cocoa_100g,code,collagen-meat-protein-ratio_100g,copper_100g,countries,countries_en,countries_tags,created_datetime,created_t,creator,emb_codes,emb_codes_tags,energy-from-fat_100g,energy_100g,fat_100g,fiber_100g,first_packaging_code_geo,fluoride_100g,fruits-vegetables-nuts_100g,generic_name,image_small_url,image_url,ingredients_from_palm_oil,ingredients_from_palm_oil_n,ingredients_from_palm_oil_tags,ingredients_text,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_tags,iodine_100g,iron_100g,labels,labels_en,labels_tags,last_modified_datetime,last_modified_t,magnesium_100g,main_category,main_category_en,manganese_100g,manufacturing_places,manufacturing_places_tags,molybdenum_100g,monounsaturated-fat_100g,no_nutriments,nucleotides_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,nutrition_grade_fr,nutrition_grade_uk,omega-3-fat_100g,omega-6-fat_100g,omega-9-fat_100g,origins,origins_tags,packaging,packaging_tags,pantothenic-acid_100g,ph_100g,phosphorus_100g,pnns_groups_1,pnns_groups_2,polyols_100g,polyunsaturated-fat_100g,potassium_100g,product_name,proteins_100g,purchase_places,quantity,salt_100g,saturated-fat_100g,selenium_100g,serum-proteins_100g,serving_size,silica_100g,sodium_100g,starch_100g,states,states_en,states_tags,stores,sugars_100g,taurine_100g,traces,traces_en,traces_tags,trans-fat_100g,url,vitamin-a_100g,vitamin-b12_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-c_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-pp_100g,zinc_100g
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ sucre -> fr:sucre ] [ farine-de-ble -> fr...,"E1400 - Dextrin,E322 - Lecithins,E503 - Ammoni...",3,"en:e1400,en:e322,en:e503",NaN,4,NaN,NaN,NaN,NaN,St Michel,st-michel,NaN,NaN,NaN,NaN,NaN,sablés,"Sugary snacks,Biscuits and cakes,Biscuits,fr:S...","en:sugary-snacks,en:biscuits-and-cakes,en:bisc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0000007730009,NaN,NaN,France,France,en:france,2016-05-09T15:07:12Z,1462806432,openfoodfacts-contributors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Biscuits sablés fourrage au cacao,http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,NaN,0,NaN,"Sucre, farine de _Blé_, graisse et huiles végé...",NaN,1,huile-vegetale,NaN,NaN,NaN,NaN,NaN,2016-05-09T16:07:17Z,1462810037,NaN,en:sugary-snacks,Sugary snacks,NaN,France,france,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carton,carton,NaN,NaN,NaN,Sugary snacks,Biscuits and cakes,NaN,NaN,NaN,Biscuits sablés fourrage au cacao,NaN,NaN,1 kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","To be completed,Nutrition facts to be complete...","en:to-be-completed,en:nutrition-facts-to-be-co...",NaN,NaN,NaN,"Fruits à coque,Soja,Graines de sésame","Nuts,Sesame seeds,Soybeans","en:nuts,en:sesame-seeds,en:soybeans",NaN,ht

In [11]:
#food_countries.groupby('countries_en', as_index=False)['allergens'].mean()
final_data = food_countries.groupby('countries_en', as_index=False)['allergens','additives_n'].mean()
final_data

,countries_en,allergens,additives_n
0,Algeria,6.000000,1.000000
1,Allemagne,1.000000,1.000000
2,Andorra,4.000000,1.000000
3,Armenia,1.000000,0.000000
4,Australia,3.090909,1.099174
...,...,...,...
61,United Kingdom,2.806653,2.126819
62,United States,4.486653,2.921971
63,de:Scotland,1.000000,0.000000
64,fr:Angleterre,2.000000,2.000000


In [15]:
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('viveksethia', 'n7aiy5bf6d')


trace1 = go.Bar(
    x=final_data['countries_en'],
    y=final_data['allergens'],
    name='Allergens'
)
trace2 = go.Bar(
    x=final_data['countries_en'],
    y=final_data['additives_n'],
    name='Additives'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~viveksethia/0 or inside your plot.ly account where it is named 'grouped-bar'
